# Confision Matrix

In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
from deepchecks.vision.dataset import VisionData
from torchvision import models
import torchvision
import torch
from torch import nn
from torchvision.transforms import ToTensor
import copy

In [3]:
from deepchecks.vision.checks.performance import ConfusionMatrixReport

In [4]:

from deepchecks.vision.datasets.classification import mnist
def simple_nn():
    torch.manual_seed(42)
    class NeuralNetwork(nn.Module):
        def __init__(self):
            super(NeuralNetwork, self).__init__()
            self.flatten = nn.Flatten()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(28 * 28, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 10)
            )

        def forward(self, x):
            x = self.flatten(x)
            logits = self.linear_relu_stack(x)
            return logits

    model = NeuralNetwork().to('cpu')
    return model

In [5]:
def trained_mnist(simple_nn, mnist_data_loader_train):
    torch.manual_seed(42)
    simple_nn = copy.deepcopy(simple_nn)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(simple_nn.parameters(), lr=1e-3)
    size = len(mnist_data_loader_train.dataset)
    # Training 1 epoch
    simple_nn.train()
    for batch, (X, y) in enumerate(mnist_data_loader_train):
        X, y = X.to('cpu'), y.to('cpu')

        # Compute prediction error
        pred = simple_nn(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    return simple_nn


In [19]:
train_ds = mnist.load_dataset(train=True, object_type='VisionData')
test_ds = mnist.load_dataset(train=False, object_type='VisionData')
model = trained_mnist(simple_nn(), train_ds.get_data_loader())


loss: 2.324835  [    0/60000]
loss: 2.264989  [ 6400/60000]
loss: 2.202413  [12800/60000]
loss: 2.143342  [19200/60000]
loss: 2.094676  [25600/60000]
loss: 2.038667  [32000/60000]
loss: 2.019060  [38400/60000]
loss: 1.949741  [44800/60000]
loss: 1.911960  [51200/60000]
loss: 1.741876  [57600/60000]


In [20]:
from deepchecks.vision.utils.classification_formatters import ClassificationPredictionFormatter
check = ConfusionMatrixReport()
check.run(train_ds, model, prediction_formatter=ClassificationPredictionFormatter(mnist.mnist_prediction_formatter))

Confusion Matrix Calculate the confusion matrix of the model on the given dataset. Read More... Additional Outputs Showing 10 of 10 classes:

# confusion matrix report object detection

In [8]:
from deepchecks.vision.datasets.detection import coco
import numpy as np

In [9]:
yolo = coco.load_model(pretrained=True)


Downloading: "https://github.com/ultralytics/yolov5/archive/v6.1.zip" to /Users/dbi/.cache/torch/hub/v6.1.zip


In [10]:
coco_dataloader = coco.load_dataset()

In [11]:
from deepchecks.vision.dataset import VisionData

In [13]:
from deepchecks.vision.utils.detection_formatters import DetectionLabelFormatter, DetectionPredictionFormatter
train_ds = VisionData(coco_dataloader, label_formatter=DetectionLabelFormatter(coco.yolo_label_formatter), num_classes=80)

In [18]:
check = ConfusionMatrixReport( categories_to_display=100)
check.run(train_ds, yolo, prediction_formatter=DetectionPredictionFormatter(coco.yolo_prediction_formatter))

81
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', 'not found']


Confusion Matrix Calculate the confusion matrix of the model on the given dataset. Read More... Additional Outputs last category are detections that do not overlap with labeled data and labels that have not been detected. Showing 100 of 80 classes: